In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
import numpy as np
import os
import sys
import time
import datetime
import glob
sys.path.append('../py/')
from s027_kfold_ods import ods_kfold
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import logger_func, get_categorical_features, get_numeric_features, reduce_mem_usage, elo_save_feature, impute_feature
try:
    if not logger:
        logger=logger_func()
except NameError:
    logger=logger_func()

from sklearn.metrics import mean_squared_error, roc_auc_score


#========================================================================
# Args
out_part = ['', 'part', 'all'][0]
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_active_month', 'index', 'personal_term', 'no_out_flg']
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())
#========================================================================

2019-02-24 22:18:34,302 utils 400 [INFO]    [logger_func] start 


In [2]:
df_hist = pd.read_csv('../input/historical_transactions.csv')
df_hist['purchase_amount_new'] = np.round(df_hist['purchase_amount'] / 0.00150265118 + 497.06, 2)

df_hist['installments'] = df_hist['installments'].map(lambda x:  
                                                    1 if x<1 else
                                                    1 if x>100 else
                                                      x
                                                     )
df_hist['authorized_flag'] = df_hist['authorized_flag'].map({'Y':1, 'N':0})
df_hist['category_1'] = df_hist['category_1'].map({'Y':1, 'N':0}) 
df_hist['yyyymm'] = df_hist['purchase_date'].map(lambda x: str(x)[:7])
df_hist['yyyymmdd'] = df_hist['purchase_date'].map(lambda x: str(x)[:10])

In [14]:
# df_first = df_hist[[key, 'purchase_date']].groupby(key)['purchase_date'].first().reset_index()
# df_first = df_first.to_frame().reset_index()
# df_first = df_first.merge(df_hist, how='inner', on=[key, 'purchase_date'])



array(['C_ID_0009010641', 'C_ID_001fc7b4d5', 'C_ID_0028ac8028',
       'C_ID_002fa5e9b3', 'C_ID_0030e0945f'], dtype=object)

In [39]:
pd.set_option('max_rows', 500)

# Get id_list
tmp = df_first[[key, 'purchase_amount_new']]
tmp = tmp[tmp['purchase_amount_new']<=3]
id_list = tmp[key].unique()

# base = utils.read_pkl_gzip('../input/base_type_group.gz')
base['outlier'] = base[target].map(lambda x: 1 if x<-30 else 0)
train = base[~base[target].isnull()]
print(train.loc[train[key].isin(id_list), target].shape)
tmp = train.loc[train[key].isin(id_list), ['group', 'outlier']]
display(tmp.groupby('group')['outlier'].sum())
display(tmp['group'].value_counts())
# tmp.map(lambda x: np.round(x, 0)).value_counts()

(4159,)


group
type0_flg0_higher    38
type0_flg0_lower      1
type0_flg1_lower      0
type10                0
type1_flg0_lower      0
type1_flg1_higher     0
type1_flg1_lower      1
type2_flg0_higher     1
type2_flg0_lower      9
type2_flg1_lower      0
Name: outlier, dtype: int64

type2_flg0_lower     1910
type0_flg0_higher     689
type10                625
type0_flg0_lower      354
type1_flg0_lower      187
type1_flg1_lower      172
type2_flg0_higher     121
type2_flg1_lower       56
type0_flg1_lower       44
type1_flg1_higher       1
Name: group, dtype: int64

In [43]:
prefix = f"269_fpa"
base.loc[base[key].isin(id_list), 'first_purchase_amount_new_less_than_3'] = 1
base['first_purchase_amount_new_less_than_3'].fillna(0, inplace=True)
elo_save_feature(df_feat=base[[key, 'first_purchase_amount_new_less_than_3']], prefix=prefix)